<a href="https://colab.research.google.com/github/zoha87222/online-exam-cheating-detection/blob/main/Online_Exam_Cheating_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import urllib.request

# Create the models directory if it doesn't exist
os.makedirs("models", exist_ok=True)

# URL of the YOLOv8n model file
url = "https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt"
destination = "models/yolov8n.pt"

# Download the model
print("⬇️  Downloading YOLOv8n model...")
urllib.request.urlretrieve(url, destination)
print("✅ YOLOv8n model downloaded and saved to models/yolov8n.pt")



⬇️  Downloading YOLOv8n model...
✅ YOLOv8n model downloaded and saved to models/yolov8n.pt


In [ ]:
!wget -q -O blaze_face_short_range.tflite https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite


In [ ]:
!wget -q -O face_landmarker_v2_with_blendshapes.task https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task


In [ ]:
!pip install mediapipe

In [ ]:
!pip install -U ultralytics

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from ultralytics import YOLO
import time
from google.colab.patches import cv2_imshow
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from IPython.display import clear_output
from google.colab import files
uploaded = files.upload()


YOLO_MODEL_PATH = 'yolov8n.pt'

if uploaded:

    VIDEO_PATH = list(uploaded.keys())[0]
    print(f"File uploaded successfully: {VIDEO_PATH}")
else:
    print("No File Uploaded")


MAX_YAW_DEG = 35
MAX_PITCH_DEG = 25


yolo_model = YOLO(YOLO_MODEL_PATH)
away_count = 0
phone_detected_count = 0
unauthorized_person_detected_count = 0
calibrated_pitch, calibrated_yaw = 0, 0
detected_object_types = set()


model_points = np.array([
    (0.0, 0.0, 0.0),
    (0.0, -330.0, -65.0),
    (-225.0, 170.0, -135.0),
    (225.0, 170.0, -135.0),
    (-150.0, -150.0, -125.0),
    (150.0, -150.0, -125.0)
], dtype=np.float64)


base_options_land = python.BaseOptions(model_asset_path='/content/face_landmarker_v2_with_blendshapes.task')
options_land = vision.FaceLandmarkerOptions(
    base_options=base_options_land,
    running_mode=vision.RunningMode.IMAGE,
    num_faces=2
)

def estimate_head_pose(landmarks, width, height):

    image_points = np.array([
        (landmarks[1].x * width, landmarks[1].y * height),
        (landmarks[152].x * width, landmarks[152].y * height),
        (landmarks[33].x * width, landmarks[33].y * height),
        (landmarks[263].x * width, landmarks[263].y * height),
        (landmarks[61].x * width, landmarks[61].y * height),
        (landmarks[291].x * width, landmarks[291].y * height)
    ], dtype=np.float64)

    focal_length = width
    center = (width / 2, height / 2)
    camera_matrix = np.array([[focal_length, 0, center[0]], [0, focal_length, center[1]], [0, 0, 1]], dtype=np.float64)
    dist_coeffs = np.zeros((4, 1))

    success, rot_vec, trans_vec = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs)
    if not success: return None

    rmat, _ = cv2.Rodrigues(rot_vec)
    angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)
    return angles

def convert_frame(frame):
    return mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))


cap = cv2.VideoCapture(VIDEO_PATH)

with vision.FaceLandmarker.create_from_options(options_land) as landmarker:


    print("Calibrating... Please look directly at the camera.")
    calibration_data = []
    while len(calibration_data) < 30:
        ret, frame = cap.read()
        if not ret: break
        mp_img = convert_frame(frame)
        res = landmarker.detect(mp_img)
        if res.face_landmarks:
            angles = estimate_head_pose(res.face_landmarks[0], frame.shape[1], frame.shape[0])
            if angles: calibration_data.append(angles[:2])

    if calibration_data:
        calibrated_pitch, calibrated_yaw = np.mean(calibration_data, axis=0)
        print(f"Calibration Complete: Pitch {calibrated_pitch:.2f}, Yaw {calibrated_yaw:.2f}")


    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        h, w, _ = frame.shape
        mp_img = convert_frame(frame)
        land_res = landmarker.detect(mp_img)


        looking_away = False
        multi_face = False


        if land_res.face_landmarks:
            num_faces = len(land_res.face_landmarks)
            if num_faces > 1:
                unauthorized_person_detected_count += 1
                multi_face = True


            angles = estimate_head_pose(land_res.face_landmarks[0], w, h)
            if angles:
                p_diff = abs(angles[0] - calibrated_pitch)
                y_diff = abs(angles[1] - calibrated_yaw)
                if p_diff > MAX_PITCH_DEG or y_diff > MAX_YAW_DEG:
                    away_count += 1
                    looking_away = True


        y_res = yolo_model(frame, verbose=False, conf=0.4)
        phone_present = False
        for r in y_res:
            for box in r.boxes:
                cls = r.names[int(box.cls[0])]
                if cls in ["cell phone", "book"]:
                    phone_detected_count += 1
                    phone_present = True
                    detected_object_types.add(cls)
                    b = box.xyxy[0].cpu().numpy().astype(int)
                    cv2.rectangle(frame, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 2)


        if looking_away: cv2.putText(frame, "GAZE VIOLATION", (30, 50), 2, 0.8, (0, 0, 255), 2)
        if multi_face: cv2.putText(frame, "MULTIPLE PEOPLE", (30, 80), 2, 0.8, (0, 0, 255), 2)
        if phone_present: cv2.putText(frame, "OBJECT PROHIBITED", (30, 110), 2, 0.8, (0, 0, 255), 2)



cap.release()
print(f"\n--- Final Proctoring Report ---")


gaze_status = "Yes" if away_count > 0 else "No"
object_status = "Yes" if phone_detected_count > 0 else "No"
person_status = "Yes" if unauthorized_person_detected_count > 0 else "No"
object_list = ", ".join(detected_object_types) if detected_object_types else "N/A"

print(f"Gaze Violation Detected: {gaze_status}")
print(f"Unauthorized Object Detected: {object_status} ({object_list})")
print(f"Multiple Person Violation: {person_status}")


NameError: name 'audio_classifier' is not defined